# Semantic Kernel med OpenBnB MCP Server-integration

Den här anteckningsboken visar hur man använder Semantic Kernel med den faktiska OpenBnB MCP-servern för att söka efter riktiga Airbnb-boenden med MCPStdioPlugin. För åtkomst till LLM används Azure AI Foundry. För att konfigurera dina miljövariabler kan du följa [Setup Lesson](/00-course-setup/README.md)


## Importera de nödvändiga paketen


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Skapa MCP Plugin-anslutningen

Vi kommer att ansluta till [OpenBnB MCP-servern](https://github.com/openbnb-org/mcp-server-airbnb) med hjälp av MCPStdioPlugin. Den här servern erbjuder sökfunktionalitet för Airbnb via @openbnb/mcp-server-airbnb-paketet.


## Skapa klienten

I det här exemplet kommer vi att använda Azure AI Foundry för vår LLM-åtkomst. Se till att dina miljövariabler är korrekt inställda.


## Miljökonfiguration

Konfigurera inställningar för Azure OpenAI. Se till att du har följande miljövariabler inställda:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Förstå OpenBnB MCP-integrationen

Den här notebooken ansluter till **den riktiga OpenBnB MCP-servern** som erbjuder faktisk sökfunktionalitet för Airbnb.

### Så här fungerar det:

1. **MCPStdioPlugin**: Använder standard in-/utgångskommunikation med MCP-servern
2. **Äkta NPM-paket**: Hämtar och kör `@openbnb/mcp-server-airbnb` via npx
3. **Live-data**: Returnerar verklig Airbnb-fastighetsdata från deras API:er
4. **Funktionsupptäckt**: Agenten upptäcker automatiskt tillgängliga funktioner från MCP-servern

### Tillgängliga funktioner:

OpenBnB MCP-servern erbjuder vanligtvis:
- **search_listings** - Sök efter Airbnb-fastigheter baserat på plats och kriterier
- **get_listing_details** - Hämta detaljerad information om specifika fastigheter
- **check_availability** - Kontrollera tillgänglighet för specifika datum
- **get_reviews** - Hämta recensioner för fastigheter
- **get_host_info** - Hämta information om fastighetsvärdar

### Förutsättningar:

- **Node.js** installerat på ditt system
- **Internetanslutning** för att ladda ner MCP-serverpaketet
- **NPX** tillgängligt (ingår med Node.js)

### Testa anslutningen:

Du kan testa MCP-servern manuellt genom att köra:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Detta kommer att ladda ner och starta OpenBnB MCP-servern, som Semantic Kernel sedan ansluter till för verklig Airbnb-data.


## Köra agenten med OpenBnB MCP-servern

Nu ska vi köra AI-agenten som ansluter till OpenBnB MCP-servern för att söka efter riktiga Airbnb-boenden i Stockholm för 2 vuxna och 1 barn. Du kan ändra listan `user_inputs` för att justera sökkriterierna.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Sammanfattning  
Grattis! Du har framgångsrikt byggt en AI-agent som integrerar med verklig boendesökning med hjälp av Model Context Protocol (MCP):

Teknologier som används:  
- Semantic Kernel – För att bygga intelligenta agenter med Azure OpenAI  
- Azure AI Foundry – För LLM-funktioner och chattkomplettering  
- MCP (Model Context Protocol) – För standardiserad verktygsintegration  
- OpenBnB MCP Server – För verklig Airbnb-sökfunktionalitet  
- Node.js/NPX – För att köra den externa MCP-servern  

Vad du har lärt dig:  
- MCP-integration: Att koppla Semantic Kernel-agenter till externa MCP-servrar  
- Realtidsdataåtkomst: Söka efter faktiska Airbnb-egenskaper via live-API:er  
- Protokollkommunikation: Använda stdio-kommunikation mellan agent och MCP-server  
- Funktionsupptäckt: Automatiskt upptäcka tillgängliga funktioner från MCP-servrar  
- Strömmande svar: Fånga och logga funktionsanrop i realtid  
- HTML-rendering: Formatera agentens svar med stiliserade tabeller och interaktiva visningar  

Nästa steg:  
- Integrera ytterligare MCP-servrar (väder, flyg, restauranger)  
- Bygga ett multi-agent-system som kombinerar MCP- och A2A-protokoll  
- Skapa anpassade MCP-servrar för dina egna datakällor  
- Implementera ihågkommen konversation över flera sessioner  
- Distribuera agenten till Azure Functions med MCP-serverorkestrering  
- Lägg till användarautentisering och bokningsfunktioner  

Viktiga fördelar med MCP-arkitektur:  
- Standardisering: Universellt protokoll för att koppla AI-agenter till externa verktyg  
- Realtidsdata: Tillgång till aktuell och uppdaterad information från olika tjänster  
- Utbyggbarhet: Enkel integration av nya datakällor och verktyg  
- Interoperabilitet: Fungerar över olika AI-ramverk och agentplattformar  
- Ansvarsseparation: Tydlig åtskillnad mellan AI-logik och extern dataåtkomst  



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiska översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess ursprungliga språk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
